<a href="https://colab.research.google.com/github/glorivaas/Risk_Measures/blob/main/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4: Elliptical distributions continued

### Exercise 1
1. Implement MLE of student's t and normal distribution below.
1. Fetch recent SPX500.
1. Calculate **5-day** Value-at-Risk of a) SPX500 value changes b) SPX500 log-returns, evaluated at **31.03.2025**.
1. Compare with the realized losses.
1. What was the probability, under four models (2x returns, 2x parametric forms), of observing losses of that or greater magnitude?
1. Which model fits best (has the highest log-likelihood)?

In [20]:
import numpy as np
from scipy.stats import norm
from scipy import stats, optimize

def fit_student_t(
    data: list[float]
) -> tuple[float, float, float]:
    """
    Fit Student's t distribution to data using MLE. Returns:
    - degrees of freedom
    - location
    - scale
    """
    mean = np.mean(data)
    std = np.std(data, ddof=0)

    def negative_log_likelihood(params):
        df, loc, scale = params
        if scale <= 0 or df <= 0:
            return np.inf  # invalid parameters
        return -np.sum(stats.t.logpdf(data, df, loc=loc, scale=scale))

    result = optimize.minimize(
        negative_log_likelihood,
        x0=[5, mean, std],  # initial guesses: df=5
        bounds=[(1e-5, None), (None, None), (1e-5, None)]  # df and scale must be >0
    )

    df, loc, scale = result.x
    return df, loc, scale

def student_t_var(
    data: list[float],
    confidence_level: float = 0.99,
) -> list[float]:
    """
    Calculate Value at Risk using Student's t distribution.
    """
    df, loc, scale = fit_student_t(data)
    var = loc + scale * stats.t.ppf(1 - confidence_level, df)
    return var

def fit_normal(
    data: list[float]
) -> tuple[float, float]:
    """
    Fit normal distribution to data using MLE. Returns:
    - mean
    - standard deviation
    """
    mean = np.mean(data)
    std = np.std(data, ddof=0)
    return mean, std

def normal_var(
    data: list[float],
    confidence_level: float = 0.99,
) -> list[float]:
    """
    Calculate Value at Risk using normal distribution.
    """
    mean, std = fit_normal(data)
    var = mean + std * norm.ppf(1 - confidence_level)
    return var

#Part 2

import yfinance as yf
import pandas as pd

def fetch_spx500_data(start_date: str, end_date: str) -> pd.DataFrame:
    spx = yf.download('^GSPC', start=start_date, end=end_date)
    return spx

spx_data = fetch_spx500_data('2025-01-01', '2025-04-05')



#Part 3

spx_close = spx_data['Close']['^GSPC']
spx_5d_value_change = spx_close - spx_close.shift(5)
spx_5d_log_return = np.log(spx_close / spx_close.shift(5))
spx_df = pd.DataFrame({
    'Close': spx_close,
    '5d_value_change': spx_5d_value_change,
    '5d_log_return': spx_5d_log_return
})

fit_data = spx_df[spx_df.index < '2025-03-31']

value_change_data = fit_data['5d_value_change'].dropna().tolist()
log_return_data = fit_data['5d_log_return'].dropna().tolist()

# Normal VaR
value_change_var_normal = normal_var(value_change_data)
log_return_var_normal = normal_var(log_return_data)

# Student's t VaR
value_change_var_student = student_t_var(value_change_data)
log_return_var_student = student_t_var(log_return_data)

print("5-day VaR for SPX500 (at 99% confidence level):")
print(f"Value Changes - Normal: {value_change_var_normal:.4f}")
print(f"Value Changes - Student's t: {value_change_var_student:.4f}")
print(f"Log Returns - Normal: {log_return_var_normal:.4f}")
print(f"Log Returns - Student's t: {log_return_var_student:.4f}")


#Part 4

realized_value_change = spx_df.loc['2025-03-31', '5d_value_change']
realized_log_return = spx_df.loc['2025-03-31', '5d_log_return']

print("\nComparing Value Changes:")
if realized_value_change < value_change_var_normal:
    print("Realized loss exceeded Normal VaR for value changes. That´s a bad result.")
else:
    print("Realized loss was within Normal VaR for value changes. ")

if realized_value_change < value_change_var_student:
    print("Realized loss exceeded Student's t VaR for value changes. That's a bad result. ")
else:
    print("Realized loss was within Student's t VaR for value changes. ")

print("\nComparing Log Returns:")
if realized_log_return < log_return_var_normal:
    print("Realized loss exceeded Normal VaR for log returns. ")
else:
    print("Realized loss was within Normal VaR for log returns. ")

if realized_log_return < log_return_var_student:
    print("Realized loss exceeded Student's t VaR for log returns. ")
else:
    print("Realized loss was within Student's t VaR for log returns. ")


#Part 5

from scipy.stats import norm, t

mean_vc, std_vc = fit_normal(value_change_data)
mean_lr, std_lr = fit_normal(log_return_data)

df_vc, loc_vc, scale_vc = fit_student_t(value_change_data)
df_lr, loc_lr, scale_lr = fit_student_t(log_return_data)

prob_value_change_normal = norm.cdf(realized_value_change, loc=mean_vc, scale=std_vc)
prob_value_change_student = t.cdf(realized_value_change, df=df_vc, loc=loc_vc, scale=scale_vc)
prob_log_return_normal = norm.cdf(realized_log_return, loc=mean_lr, scale=std_lr)
prob_log_return_student = t.cdf(realized_log_return, df=df_lr, loc=loc_lr, scale=scale_lr)

print("\nProbability of realizing a loss bigger than observed (Left-Tail Probability):")
print(f"Value Changes - Normal model: {prob_value_change_normal:.6f}")
print(f"Value Changes - Student's t model: {prob_value_change_student:.6f}")
print(f"Log Returns - Normal model: {prob_log_return_normal:.6f}")
print(f"Log Returns - Student's t model: {prob_log_return_student:.6f}")


#Part 6

loglik_value_change_normal = np.sum(norm.logpdf(value_change_data, loc=mean_vc, scale=std_vc))
loglik_value_change_student = np.sum(t.logpdf(value_change_data, df=df_vc, loc=loc_vc, scale=scale_vc))
loglik_log_return_normal = np.sum(norm.logpdf(log_return_data, loc=mean_lr, scale=std_lr))
loglik_log_return_student = np.sum(t.logpdf(log_return_data, df=df_lr, loc=loc_lr, scale=scale_lr))

print("\nLog-Likelihoods:")
print(f"Value Changes - Normal model: {loglik_value_change_normal:.2f}")
print(f"Value Changes - Student's t model: {loglik_value_change_student:.2f}")
print(f"Log Returns - Normal model: {loglik_log_return_normal:.2f}")
print(f"Log Returns - Student's t model: {loglik_log_return_student:.2f}")


[*********************100%***********************]  1 of 1 completed


5-day VaR for SPX500 (at 99% confidence level):
Value Changes - Normal: -305.1465
Value Changes - Student's t: -306.0429
Log Returns - Normal: -0.0521
Log Returns - Student's t: -0.0523

Comparing Value Changes:
Realized loss was within Normal VaR for value changes. 
Realized loss was within Student's t VaR for value changes. 

Comparing Log Returns:
Realized loss was within Normal VaR for log returns. 
Realized loss was within Student's t VaR for log returns. 

Probability of realizing a loss bigger than observed (Left-Tail Probability):
Value Changes - Normal model: 0.134142
Value Changes - Student's t model: 0.134070
Log Returns - Normal model: 0.126325
Log Returns - Student's t model: 0.126174

Log-Likelihoods:
Value Changes - Normal model: -336.28
Value Changes - Student's t model: -336.31
Log Returns - Normal model: 132.23
Log Returns - Student's t model: 132.19


For part 6 we have know that the higher log-likelihood the better.

For both value changes and log returns, Normal and Student’s t log-likelihoods are very close, Normal slightly better.

-----

$X$ is said to have multidimensional t distribution with mean $\mu$, dispersion $\Sigma$ and $v$ degrees of freedom, $X\sim t_d(\mu,\Sigma,v)$, if
$$
    X \overset{d}{=} Y\sqrt{\frac{v}{U}} + \mu,
$$
where $Y\sim\mathcal{N}(0,\Sigma)$ and $U\sim \chi_v^2$ are independent.

### Exercise 2
1. Implement a method of fitting multidimensional t distribution to data, can be MLE, EM Algorithm or some other.
1. Choose selection of 20 SPX500 stocks (can be the same as in the previous notebook) and calculate daily log-returns.
1. Calculate VaR for a linear portfolio of those stocks and perform backtesting.

In [33]:
import numpy as np
import numpy.typing as npt
import pandas as pd
import yfinance as yf
from scipy.stats import t, chi2
from scipy.optimize import minimize

def fit_t_distribution(data: list[float]) -> tuple[float, npt.NDArray[np.double], npt.NDArray[np.double]]:
    """
    Fit t distribution to data. Returns:
    - degrees of freedom
    - mean
    - dispersion matrix
    """
    data = np.asarray(data)

    mean = np.mean(data, axis=0)
    dispersion = np.cov(data, rowvar=False)

    dfs = []
    for i in range(data.shape[1]):
        _, loc, scale = t.fit(data[:, i])
        df, _, _ = t.fit(data[:, i])
        dfs.append(df)
    df_estimate = np.mean(dfs)

    return df_estimate, mean, dispersion

def var_t_distribution(data: list[float], confidence_level: float = 0.99) -> float:
    """
    Calculate Value at Risk using multivariate t-distribution.
    """
    df, mean, dispersion = fit_t_distribution(data)

    if isinstance(dispersion, np.ndarray):
        d = dispersion.shape[0]
    else:
        d = 1
    q = chi2.ppf(confidence_level, d)

    scaling = np.sqrt((df - 2) / df)

    var = scaling * np.sqrt(q)
    return var


#Part 2
tickers = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'TSLA', 'JPM', 'V', 'JNJ',
    'WMT', 'PG', 'MA', 'UNH', 'HD', 'BAC', 'XOM', 'PFE', 'DIS', 'KO'
]

full_data = yf.download(tickers, start='2025-01-01', end='2025-04-01')

# Use Close prices (safe)
close_prices = full_data['Close']

# Calculate daily log-returns
log_returns = np.log(close_prices / close_prices.shift(1)).dropna()

print("\nSample of daily log-returns:")
print(log_returns.head())

# =========================
# STEP 3 - Fit Multivariate t
# =========================

df_t, mean_t, cov_t = fit_t_distribution(log_returns.values)

print(f"\nEstimated degrees of freedom: {df_t:.2f}")
print(f"Mean vector shape: {mean_t.shape}")
print(f"Covariance matrix shape: {cov_t.shape}")

# =========================
# STEP 4 - Build Portfolio
# =========================

n_assets = log_returns.shape[1]

# Equal weights portfolio
weights = np.ones(n_assets) / n_assets

# Portfolio statistics
portfolio_mean = weights @ mean_t
portfolio_variance = weights @ cov_t @ weights
portfolio_std = np.sqrt(portfolio_variance)

# =========================
# STEP 5 - Calculate Portfolio VaR
# =========================

# Chi-squared quantile for 1 degree of freedom
q = chi2.ppf(0.99, 1)
scaling = np.sqrt((df_t - 2) / df_t)

# Portfolio VaR (negative for loss)
portfolio_var = -(portfolio_mean + scaling * np.sqrt(q) * portfolio_std)

print(f"\nPortfolio 1-day 99% VaR based on Multivariate Student's t: {portfolio_var:.6f}")

# =========================
# STEP 6 - Backtesting
# =========================

# Calculate actual portfolio returns
portfolio_returns = log_returns @ weights

# Count violations (loss exceeds VaR)
violations = (portfolio_returns < -portfolio_var).sum()
total_days = len(portfolio_returns)
violation_rate = violations / total_days

print(f"\nBacktesting results:")
print(f"Total days: {total_days}")
print(f"Number of violations (loss > VaR): {violations}")
print(f"Violation rate: {violation_rate:.4f} (Expected ~0.01 for 99% VaR)")

[*********************100%***********************]  20 of 20 completed



Sample of daily log-returns:
Ticker          AAPL      AMZN       BAC       DIS     GOOGL        HD  \
Date                                                                     
2025-01-03 -0.002011  0.017867  0.011672  0.003063  0.012381  0.001852   
2025-01-06  0.006716  0.015140  0.013081 -0.000990  0.026143  0.000488   
2025-01-07 -0.011453 -0.024461  0.014867  0.003057 -0.007034 -0.013289   
2025-01-08  0.002021  0.000090  0.002817 -0.014741 -0.007909  0.007700   
2025-01-10 -0.024399 -0.014465 -0.024092 -0.010164 -0.009897  0.006923   

Ticker           JNJ       JPM        KO        MA      META      MSFT  \
Date                                                                     
2025-01-03  0.001180  0.013574 -0.001456 -0.001993  0.008954  0.011331   
2025-01-06 -0.003682 -0.004887 -0.015340 -0.018253  0.041421  0.010573   
2025-01-07  0.017731  0.009586  0.000493 -0.001036 -0.019727 -0.012891   
2025-01-08 -0.027454 -0.000164  0.014199  0.009730 -0.011672  0.005172   
2025-01